## Go through all steps I learned so far
take care to the path!!

In [1]:
import os
'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: notebooks'

## step 1. update data

In [2]:
# %load ../src/data/get_data.py

# check if the path is in notebooks, if not, adjust the os.path.dirname()!!!

import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def get_johns_hopkins():
    ''' Get data by a git pull request, the source code has to be pulled first
        Result is stored in the predifined csv structure
    '''
    git_pull = subprocess.Popen( "/usr/bin/git pull" ,
                         cwd = os.path.dirname( '../data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))


def get_current_data_germany():
    ''' Get current data from germany, attention API endpoint not too stable
        Result data frame is stored as pd.DataFrame

    '''
    # 16 states
    #data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    # 400 regions / Landkreise
    data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    json_object=json.loads(data.content)
    full_list=[]
    for pos,each_dict in enumerate (json_object['features'][:]):
        full_list.append(each_dict['attributes'])

    pd_full_list=pd.DataFrame(full_list)
    pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')
    print(' Number of regions rows: '+str(pd_full_list.shape[0]))

if __name__ == '__main__':
    get_johns_hopkins()
    get_current_data_germany()


Error : b'From https://github.com/CSSEGISandData/COVID-19\n   31b109c6..e28775e6  web-data   -> origin/web-data\n'
out : b'Already up to date.\n'
 Number of regions rows: 412


## step 2 Process data to relational form

In [10]:
# %load ../src/data/process_JH_data.py

# remember checking the path!!!

import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    ''' Transformes the COVID data in a relational data set

    '''

    data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    print(' Latest date is: '+str(max(pd_relational_model.date)))

if __name__ == '__main__':

    store_relational_JH_data()

 Number of rows stored: 46816
 Latest date is: 2020-07-15 00:00:00


## step 3 do filtering and doubling rate calculation

In [11]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    # regression window is 3 days
    
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    # make data smooth
    
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    #pd.rolling will go through datas by a window
    
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    # take care for merging the datas!!   left vs. outer
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['country']=='Germany'].tail())


the test slope is: [2.]
            date state  country  confirmed  confirmed_filtered  confirmed_DR  \
25515 2020-07-11    no  Germany   199709.0            199628.2    563.128060   
25516 2020-07-12    no  Germany   199919.0            199919.2    680.249858   
25517 2020-07-13    no  Germany   200180.0            200230.8    848.985138   
25518 2020-07-14    no  Germany   200456.0            200520.7    745.567970   
25519 2020-07-15    no  Germany   200890.0            200810.6    564.813146   

       confirmed_filtered_DR  
25515             652.030313  
25516             679.926658  
25517             663.544861  
25518             665.747520  
25519             691.689203  


## step 4  SIR data

In [12]:
from scipy import optimize
from scipy import integrate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

#prepare data

data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

time_idx = pd_raw.columns[4:]

df_plot = pd.DataFrame({'date':time_idx})

country_list=pd_raw['Country/Region'].unique()

for each in country_list:
    df_plot[each]=np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))

time_idx = [datetime.strptime(i,"%m/%d/%y") for i in df_plot['date']] #from string to dateform
time_str = [i.strftime('%Y-%m-%d') for i in time_idx] # dateform to strings
df_plot['date']=time_idx
df_plot.to_csv('../data/processed/COVID_flat_table.csv',sep=';',index=False)


# Visualization

In [ ]:
# %load ../src/visualization/visualize.py
import pandas as pd
import numpy as np



import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())
df_input_large=pd.read_csv('../data/processed/COVID_final_set.csv',sep=';')
df_analyse=pd.read_csv('../data/processed/COVID_flat_table.csv',sep=';')


fig1 = go.Figure()
fig2 = go.Figure()


app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['US', 'Germany','Taiwan*'], # which are pre-selected
        multi=True # multi choise
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig1, id='main_window_slope'),
    
    dcc.Markdown('''
        ## Select country for SIR model prediction
        '''),
    
    dcc.Dropdown(
        id='country_drop_down_SIR',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
#         value=['Germany'], # which are pre-selected
        multi=False # multi choise
    ),
    
    dcc.Graph(figure=fig2, id='SIR_figure')
])



# make website work

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling): # input (contry_drop_donw, doubling_time) from droup down's id


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }

    
    
    # drawing
    traces = []
    for each in country_list:
        
        #slice the country for drawing
        df_plot=df_input_large[df_input_large['country']==each]
        
        
        # only plot according to country, so groupby "country"
        # and if we calculate DB rate, we use mean to approximate the DP value in the country.
        # if we need confirmed number, then do sum over all state. 
        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

@app.callback(
    Output('SIR_figure', 'figure'),
    [Input('country_drop_down_SIR', 'value')])
def SIR_figure(country_SIR):
    ydata = np.array(df_analyse.loc[:,country_SIR])
    ydata = ydata[ydata>50]
    
    t=np.arange(len(ydata)) # should make t a pd!!!!!!
    
    N0=50000000 #max susceptible population
    # ensure re-initialization 
    I0=ydata[0]
    S0=N0-I0
    R0=0
    beta=0.4
    gamma=0.1
    def SIR_model_t(SIR,t,beta,gamma):
        S,I,R=SIR
        dS_dt=-beta*S*I/N0
        dI_dt=beta*S*I/N0-gamma*I
        dR_dt=gamma*I
        return dS_dt,dI_dt,dR_dt
    def fit_odeint(x, beta, gamma):
        return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1]
    popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
    perr = np.sqrt(np.diag(pcov))
    fitted=fit_odeint(t, *popt)
    
    fitted=pd.DataFrame({'y':fitted})
    ydata = pd.DataFrame({'y':ydata})
    t=pd.DataFrame({'date':t})
    
    traces=[]
    traces.append(dict(x=t.date,
                       y=ydata.y,
                       mode='markers+lines',
                       opacity=0.9,
                       name = 'Original'
                        )
                )
    traces.append(dict(x=t.date,
                       y=fitted.y,
                       mode='lines',
                       opacity=0.9,
                       name = 'SIR model'
                       
                        )
                )
    
    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                title= f'beta = {popt[0]}  gamma = {popt[1]}  Basic Reproduction Number R0 {popt[0]/popt[1]}',
                xaxis={'title':'Days'},

                yaxis={'title':'Population infected'
                      }
        )
    }
    
if __name__ == '__main__':

    app.run_server(use_reloader=False,debug=True)


/Users/chengyihsuan/Documents/github/2020DS_covid/ds_covid19/notebooks
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
Debugger PIN: 887-921-924
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:163: RuntimeWarning:

overflow encountered in double_scalars

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:164: RuntimeWarning:

overflow encountered in double_scalars

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:165: RuntimeWarning:

overflow encountered in double_scalars

